In [22]:
import os
CANDIDATE = "kamal_preet"
ROOT = f"ds_{CANDIDATE}"
CSV_DIR = os.path.join(ROOT, "csv_files")
OUT_DIR = os.path.join(ROOT, "outputs")
NB_DIR = os.path.join(ROOT, "notebooks")
for p in [ROOT, CSV_DIR, OUT_DIR, NB_DIR]: os.makedirs(p, exist_ok=True)
print("Created:", ROOT)

Created: ds_kamal_preet


In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
from google.colab import files
uploaded = files.upload()


Saving historical_data.csv to historical_data (1).csv


In [25]:
from google.colab import files
uploaded = files.upload()

Saving fear_greed_index.csv to fear_greed_index (1).csv


In [26]:
!pip -q install reportlab scipy

In [27]:
import pandas as pd
import numpy as np
import os

hist_path = "historical_data.csv"
fg_path   = "fear_greed_index.csv"

hist = pd.read_csv(hist_path, low_memory=False)
fg   = pd.read_csv(fg_path, low_memory=False)

print("Historical rows,cols:", hist.shape)
print(hist.head(3))
print("\nFearGreed rows,cols:", fg.shape)
print(fg.head(3))

Historical rows,cols: (211224, 16)
                                      Account  Coin  Execution Price  \
0  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9769   
1  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9800   
2  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9855   

   Size Tokens  Size USD Side     Timestamp IST  Start Position Direction  \
0       986.87   7872.16  BUY  02-12-2024 22:50        0.000000       Buy   
1        16.00    127.68  BUY  02-12-2024 22:50      986.524596       Buy   
2       144.09   1150.63  BUY  02-12-2024 22:50     1002.518996       Buy   

   Closed PnL                                   Transaction Hash     Order ID  \
0         0.0  0xec09451986a1874e3a980418412fcd0201f500c95bac...  52017706630   
1         0.0  0xec09451986a1874e3a980418412fcd0201f500c95bac...  52017706630   
2         0.0  0xec09451986a1874e3a980418412fcd0201f500c95bac...  52017706630   

   Crossed       Fee      Trade ID     Tim

In [28]:
#Standardize column names
def to_snake(s):
    return s.strip().lower().replace(" ", "_").replace("-", "_").replace("/", "_")

hist.columns = [to_snake(c) for c in hist.columns]
fg.columns   = [to_snake(c) for c in fg.columns]


In [29]:
#Parse timestamps
import pytz
IST = pytz.timezone("Asia/Kolkata")

# Historical data: prefer 'timestamp' epoch ms; fallback to 'timestamp_ist' string
if "timestamp" in hist.columns:
    hist["timestamp_ms"] = pd.to_numeric(hist["timestamp"], errors="coerce").astype("Int64")
    hist["ts_utc"] = pd.to_datetime(hist["timestamp_ms"], unit="ms", utc=True, errors="coerce")
    hist["ts_ist"] = hist["ts_utc"].dt.tz_convert(IST)
    hist["date_ist"] = hist["ts_ist"].dt.date
elif "timestamp_ist" in hist.columns:
    hist["ts_ist"] = pd.to_datetime(hist["timestamp_ist"], format="%d-%m-%Y %H:%M", errors="coerce").dt.tz_localize(IST)
    hist["ts_utc"] = hist["ts_ist"].dt.tz_convert("UTC")
    hist["date_ist"] = hist["ts_ist"].dt.date
else:
    raise Exception("No timestamp found in historical file")

# Fear & Greed: prefer 'date' column, else convert epoch seconds
if "date" in fg.columns and fg["date"].notna().any():
    fg["date_ist"] = pd.to_datetime(fg["date"], errors="coerce").dt.date
elif "timestamp" in fg.columns:
    fg["_ts"] = pd.to_numeric(fg["timestamp"], errors="coerce").astype("Int64")
    fg["_utc"] = pd.to_datetime(fg["_ts"], unit="s", utc=True, errors="coerce")
    fg["date_ist"] = fg["_utc"].dt.tz_convert(IST).dt.date
else:
    raise Exception("No date/timestamp found in fear greed file")

In [30]:
#Clean essential fields (drop rows missing critical info)
essential = [c for c in ["account","coin","ts_utc","size_usd","execution_price"] if c in hist.columns]
hist = hist.dropna(subset=essential)
hist = hist.drop_duplicates()
print("After cleaning:", hist.shape)


After cleaning: (211224, 20)


In [31]:
#Normalize text fields
def normalize_side(s):
    if pd.isna(s): return np.nan
    s = str(s).strip().upper()
    if s in {"B","BUY","LONG"}: return "BUY"
    if s in {"S","SELL","SHORT"}: return "SELL"
    return s

if "side" in hist.columns:
    hist["side"] = hist["side"].apply(normalize_side)

In [32]:
#Save cleaned CSVs
hist.to_csv(os.path.join(CSV_DIR, "historical_data_clean.csv"), index=False)
fg.to_csv(os.path.join(CSV_DIR, "fear_greed_index_clean.csv"), index=False)

In [33]:
#Merge trade data with sentiment by date
# Normalize FG classification (map extreme variants into Fear/Greed)
hist["date_ist"] = pd.to_datetime(hist["date_ist"], errors="coerce").dt.date
fg["date_ist"]   = pd.to_datetime(fg["date_ist"], errors="coerce").dt.date

# 2. Normalize sentiment classification
fg["classification_norm"] = fg["classification"].astype(str).str.strip().str.lower()
fg_map = {
    "extreme fear": "Fear",
    "fear": "Fear",
    "neutral": "Neutral",
    "greed": "Greed",
    "extreme greed": "Greed"
}
fg["fg_class"] = fg["classification_norm"].map(fg_map).fillna("Neutral")

# 3. Prepare Fear & Greed subset for merge
fg_join = fg[["date_ist", "value", "fg_class"]].rename(columns={"value": "fg_value"})

# 4. Merge with historical trades
merged = hist.merge(fg_join, on="date_ist", how="left")

# 5. Save merged file
merged.to_csv(os.path.join(CSV_DIR, "trades_with_sentiment.csv"), index=False)

# 6. Quick sanity check
print("Merged rows:", merged.shape)
print("Sentiment counts:\n", merged["fg_class"].value_counts(dropna=False))

Merged rows: (211224, 22)
Sentiment counts:
 fg_class
Neutral    141012
Greed       43251
NaN         26961
Name: count, dtype: int64


In [39]:
import matplotlib.pyplot as plt
import os

# Make sure merged has fg_class
if "fg_class" not in merged.columns:
    raise ValueError("fg_class not found in merged dataset. Re-run merge cell first.")

# Group by sentiment
by_sent = merged.groupby("fg_class").agg(
    trades=("account", "count"),
    usd_volume=("size_usd", "sum"),
    win_rate=("closed_pnl", lambda s: (s > 0).mean() if s.notna().any() else np.nan),
    median_pnl=("closed_pnl", "median")
).reset_index()

print("Sentiment summary:\n", by_sent)

# ---- Chart 1: Win Rate by Sentiment ----
plt.figure()
plt.bar(by_sent["fg_class"], by_sent["win_rate"])
plt.title("Win Rate by Sentiment")
plt.xlabel("Sentiment")
plt.ylabel("Win Rate")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "winrate_by_sentiment.png"), dpi=150)
plt.close()

# ---- Chart 2: USD Volume by Sentiment ----
plt.figure()
plt.bar(by_sent["fg_class"], by_sent["usd_volume"])
plt.title("USD Volume by Sentiment")
plt.xlabel("Sentiment")
plt.ylabel("USD Volume")
plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR, "volume_by_sentiment.png"), dpi=150)
plt.close()

print("Saved sentiment charts in:", OUT_DIR)
print(os.listdir(OUT_DIR))

Sentiment summary:
   fg_class  trades    usd_volume  win_rate  median_pnl
0    Greed   43251  1.549104e+08  0.453492         0.0
1  Neutral  141012  7.260017e+08  0.410185         0.0
Saved sentiment charts in: ds_kamal_preet/outputs
['daily_usd_volume.png', 'winrate_by_sentiment.png', 'daily_trades_count.png', 'pnl_distribution.png', 'volume_by_sentiment.png']


In [34]:
# 1. Coverage
print("Time range:", merged["ts_utc"].min(), "→", merged["ts_utc"].max())

# 2. Daily summary
merged["date_ist"] = pd.to_datetime(merged["date_ist"])
daily = merged.groupby("date_ist").agg(
    trades=("account","count"),
    usd_volume=("size_usd","sum"),
    median_trade_usd=("size_usd","median")
).reset_index()
daily.to_csv(os.path.join(CSV_DIR,"daily_activity_summary.csv"), index=False)

# 3. Per coin
coin = merged.groupby("coin").agg(
    trades=("account","count"),
    usd_volume=("size_usd","sum")
).reset_index().sort_values("usd_volume", ascending=False)
coin.head(10).to_csv(os.path.join(CSV_DIR,"top_coins_by_volume.csv"), index=False)

# 4. Profitability
if "closed_pnl" in merged.columns:
    merged["closed_pnl"] = pd.to_numeric(merged["closed_pnl"], errors="coerce")
    win_rate = (merged["closed_pnl"] > 0).mean()
    median_pnl = merged["closed_pnl"].median()
    print("Win rate:", win_rate, "Median PnL:", median_pnl)

# 5. Sentiment summary
sent = merged.groupby("fg_class").agg(
    trades=("account","count"),
    usd_volume=("size_usd","sum"),
    win_rate=("closed_pnl", lambda s: (s>0).mean() if s.notna().any() else np.nan),
    median_pnl=("closed_pnl","median")
).reset_index()
sent.to_csv(os.path.join(CSV_DIR,"sentiment_summary.csv"), index=False)
print(sent)

Time range: 2023-03-28 10:40:00+00:00 → 2025-06-15 15:06:40+00:00
Win rate: 0.4112648183918494 Median PnL: 0.0
  fg_class  trades    usd_volume  win_rate  median_pnl
0    Greed   43251  1.549104e+08  0.453492         0.0
1  Neutral  141012  7.260017e+08  0.410185         0.0


In [35]:
import matplotlib.pyplot as plt

plt.figure(); plt.plot(daily["date_ist"], daily["trades"]); plt.title("Daily Trades"); plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR,"daily_trades_count.png"), dpi=150); plt.close()

plt.figure(); plt.plot(daily["date_ist"], daily["usd_volume"]); plt.title("Daily USD Volume"); plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR,"daily_usd_volume.png"), dpi=150); plt.close()

plt.figure(); plt.hist(merged["closed_pnl"].dropna(), bins=50); plt.title("PnL Distribution"); plt.tight_layout()
plt.savefig(os.path.join(OUT_DIR,"pnl_distribution.png"), dpi=150); plt.close()


In [36]:
#Basic statistical check
from scipy import stats
fear_pnl = merged.loc[merged["fg_class"]=="Fear", "closed_pnl"].dropna()
greed_pnl = merged.loc[merged["fg_class"]=="Greed", "closed_pnl"].dropna()
if len(fear_pnl)>10 and len(greed_pnl)>10:
    u, p = stats.mannwhitneyu(fear_pnl, greed_pnl, alternative="two-sided")
    print("Mann-Whitney U p-value:", p)

In [37]:
#Create a short PDF report
from reportlab.lib.pagesizes import A4
from reportlab.pdfgen import canvas
from reportlab.lib.utils import ImageReader

REPORT_PATH = os.path.join(ROOT, "ds_report.pdf")
c = canvas.Canvas(REPORT_PATH, pagesize=A4)
c.setFont("Helvetica-Bold", 14)
c.drawString(40, 800, f"Data Science Assessment — {CANDIDATE}")
c.setFont("Helvetica", 10)
y = 770
c.drawString(40, y, f"Rows (cleaned): {len(merged):,}")
y -= 15
c.drawString(40, y, f"Time range: {merged['ts_utc'].min()} -> {merged['ts_utc'].max()}")
y -= 25
# Insert one image example
img_path = os.path.join(OUT_DIR, "daily_usd_volume.png")
if os.path.exists(img_path):
    img = ImageReader(img_path)
    c.drawImage(img, 40, 420, width=500, height=300)
c.save()
print("Saved report:", REPORT_PATH)


Saved report: ds_kamal_preet/ds_report.pdf


In [41]:
import os

CANDIDATE = "kamal_preet"   # <-- put your name here (use lowercase, underscores)
COLAB_LINK = "https://colab.research.google.com/drive/your-notebook-link"  # <-- paste your Colab share link

ROOT = f"ds_{CANDIDATE}"
os.makedirs(ROOT, exist_ok=True)

readme_text = f"""
# ds_{CANDIDATE}

## Project
**Title:** Web3 Trading — Sentiment Alignment Analysis
**Candidate:** {CANDIDATE}

## Objective
Analyze trader behaviour (trade volume, profitability) and align it with the Fear & Greed index. Produce cleaned datasets, EDA charts, and a concise PDF report.

## Repository structure
- csv_files/: cleaned CSVs and summaries
- outputs/: PNG charts from EDA and sentiment
- notebooks/: Colab notebook used for analysis
- ds_report.pdf: short report with findings
- README.md

## How to run (Google Colab)
1. Open the notebook: {COLAB_LINK}
2. Upload `historical_data.csv` and `fear_greed_index.csv` when prompted.
3. Run **Runtime → Restart and run all**.
4. Results will be saved in `csv_files/`, `outputs/`, and `ds_report.pdf`.

## Key files
- historical_data_clean.csv
- fear_greed_index_clean.csv
- trades_with_sentiment.csv
- daily_activity_summary.csv
- sentiment_summary.csv
- winrate_by_sentiment.png
- volume_by_sentiment.png
- ds_report.pdf

## Key findings (fill in)
- Win rate by sentiment: [ADD YOUR RESULT]
- Median PnL overall: [ADD YOUR RESULT]

"""

with open(os.path.join(ROOT, "README.md"), "w") as f:
    f.write(readme_text.strip())

print("README.md created at:", os.path.join(ROOT, "README.md"))


README.md created at: ds_kamal_preet/README.md


In [42]:
# Replace with your name
CANDIDATE = "kamal_preet"

# 1. Zip the entire folder
!zip -r ds_{CANDIDATE}.zip ds_{CANDIDATE}

# 2. Download the zip file to your computer
from google.colab import files
files.download(f"ds_{CANDIDATE}.zip")

updating: ds_kamal_preet/ (stored 0%)
updating: ds_kamal_preet/outputs/ (stored 0%)
updating: ds_kamal_preet/outputs/daily_usd_volume.png (deflated 12%)
updating: ds_kamal_preet/outputs/daily_trades_count.png (deflated 14%)
updating: ds_kamal_preet/outputs/pnl_distribution.png (deflated 32%)
updating: ds_kamal_preet/notebooks/ (stored 0%)
updating: ds_kamal_preet/csv_files/ (stored 0%)
updating: ds_kamal_preet/csv_files/sentiment_summary.csv (deflated 17%)
updating: ds_kamal_preet/csv_files/fear_greed_index_clean.csv (deflated 77%)
updating: ds_kamal_preet/csv_files/daily_activity_summary.csv (deflated 38%)
updating: ds_kamal_preet/csv_files/top_coins_by_volume.csv (deflated 22%)
updating: ds_kamal_preet/csv_files/trades_with_sentiment.csv (deflated 86%)
updating: ds_kamal_preet/csv_files/historical_data_clean.csv (deflated 86%)
updating: ds_kamal_preet/ds_report.pdf (deflated 23%)
updating: ds_kamal_preet/outputs/winrate_by_sentiment.png (deflated 26%)
updating: ds_kamal_preet/outputs

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>